# Part I - Accident and Emergency Department Synthetic Admission Data Analysis
## by Generoso Roberto

## Introduction

Analysis of health care data has a huge potential to improve health services and patient outcomes but practicing and creating analytical tools and processes is tricky since patient privacy needs to be preserved. The dataset that is used is the A&E Synthetic Data which created as a result of an NHS England pilot project to share more data without sacrifising the privacy of patients. The project used statistical models to that tries create datasets that retains the value of the original data but can not be used to re-identify specific individual records. 

This provides an opportunity for us to get to look and analyze the A&E data that NHS collects. For this analysis the focus is on admitted cases in just one hospital.

The columns in the dataset are the following:
* `IMD_Decile_From_LSOA` - measure of deprivation. A score of 1 indicates the entry comes from the 10% most deprived area in England
* `Age_Band` - The interval of the age of the patient
* `Sex` - Coded in 1 and 2 to represent the two sexes
* `AE_Arrive_Date` - The date the patient arrived at the A&E provider
* `AE_Arrive_HourOfDay` - The interval of what time the patient arrived at the A&E
* `AE_Time_Mins` - Total number of minutes the patient stayed in the A&E
* `AE_HRG` - Code for the health reference group
* `AE_Num_Diagnoses` - Number of diagnoses the patient have in the A&E
* `AE_Num_Investigations` - Number of investigations done to the patient
* `AE_Num_Treatments` - Number of treatments done to the patient
* `AE_Arrival_Mode` - Code for the mode by which  patient arrived
* `Provider_Patient_Distance_Miles` - Distance from the patient place of origin to the provider in miles
* `Admission_Method` - Code for the method how the patient was admitted
* `ICD10_Chapter_Code` - International classification of disease chapter code
* `Treatment_Function_Code` - The treatment function under which the patient is treated.
* `Length_Of_Stay_Days` - The number of days the patient was admitted

## Preliminary Data Wrangling

In [89]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [90]:
#Load the dataset
aedata_df_original=pd.read_csv('AnE_Synthetic_Data_Provider_Admission.csv')
aedata_df = aedata_df_original

In [91]:
# High-level overview of data shape and composition
print(aedata_df.shape)
print(aedata_df.info())
print(aedata_df.head(10))

(326444, 16)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326444 entries, 0 to 326443
Data columns (total 16 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   IMD_Decile_From_LSOA             326444 non-null  int64  
 1   Age_Band                         326444 non-null  object 
 2   Sex                              326444 non-null  int64  
 3   AE_Arrive_Date                   326444 non-null  object 
 4   AE_Arrive_HourOfDay              326444 non-null  object 
 5   AE_Time_Mins                     326444 non-null  float64
 6   AE_HRG                           326372 non-null  object 
 7   AE_Num_Diagnoses                 326444 non-null  int64  
 8   AE_Num_Investigations            326444 non-null  int64  
 9   AE_Num_Treatments                326444 non-null  int64  
 10  AE_Arrival_Mode                  326444 non-null  int64  
 11  Provider_Patient_Distance_Miles  326444 non-null  fl

1. `AE_Arrive_Date` is an object and should be made into a datetime and extract the year and month
2. The following columns are categorical types
    * `Sex`
    * `AE_Arrival_Mode`
    * `Admission_Method`
    * `ICD10_Chapter_Code`
    * `Treatment_Function_Code`
2. The following columns are ordered categorical types
    * `IMD_Decile_From_LSOA`
    * `Age_Band`
    * `AE_Arrive_HourOfDay`
    * `AE_HRG`
3. The following columns have null values:
    * `AE_HRG`
    * `ICD10_Chapter_Code`
    * `Treatment_Function_Code`
    * `Length_Of_Stay_Days`

In [92]:
# 1. Convert 'AE_Arrive_Date' to datetime
aedata_df['AE_Arrive_Date']=pd.to_datetime(aedata_df['AE_Arrive_Date'])
# Check the dtype of 'AE_Arrive_Date'
print(f"The datatype of the column AE_Arrive_Date is {aedata_df['AE_Arrive_Date'].dtype}")
# Extract year and month
aedata_df['Year_Month'] = aedata_df['AE_Arrive_Date'].dt.to_period('M')
# Check the Year_Month column
aedata_df['Year_Month']

The datatype of the column AE_Arrive_Date is datetime64[ns]


0         2015-05
1         2017-09
2         2016-09
3         2014-03
4         2017-11
           ...   
326439    2017-11
326440    2017-05
326441    2015-07
326442    2016-08
326443    2017-08
Name: Year_Month, Length: 326444, dtype: period[M]

In [93]:
# 2. Convert columns into categorical type
# Create a function that converts columns into category
def covert_to_categorical(df, column_list):
    for column in column_list:
        df[column] = df[column].astype('category')
# Create a list of columns to be made into category
categorical_columns = [\
    'Sex',
    'AE_Arrival_Mode',
    'Admission_Method',
    'ICD10_Chapter_Code',
    'Treatment_Function_Code'
    ]
# Convert columns into category
covert_to_categorical(aedata_df,categorical_columns)
# Check the dtypes
aedata_df[categorical_columns].dtypes

Sex                        category
AE_Arrival_Mode            category
Admission_Method           category
ICD10_Chapter_Code         category
Treatment_Function_Code    category
dtype: object

In [94]:
# 3. Convert column to categorical types
# Create a list of columns to be made into ordered categorical data
ordinal_columns = [\
    'IMD_Decile_From_LSOA',
    'Age_Band',
    'AE_Arrive_HourOfDay',
    'AE_HRG'
    ]
# Check the different values of the columns to be changed into ordered category
# Create a function to print the unique values
def check_unique_values(df, column_list):
    for column in column_list:
        print(f'{column} : {df[column].unique()}')
# Check the unique values of the identified ordered categorical types
check_unique_values(aedata_df, ordinal_columns)

IMD_Decile_From_LSOA : [ 2  3  1  4  8  9  5  7  6 10]
Age_Band : ['1-17' '65-84' '85+' '18-24' '45-64' '25-44']
AE_Arrive_HourOfDay : ['17-20' '13-16' '05-08' '21-24' '09-12' '01-04']
AE_HRG : ['Nothing' 'Low' 'High' 'Medium' nan]


In [95]:
# Convert the ordered_categorical_columns into category type
covert_to_categorical(aedata_df,ordinal_columns)
# Create a dictionary that shows the order of the values
ordinal_columns_dict={\
    'IMD_Decile_From_LSOA': [1,2,3,4,5,6,7,8,9,10],
    'Age_Band':['1-17','18-24','25-44', '45-64', '65-84', '85+'],
    'AE_Arrive_HourOfDay':['01-04','05-08','09-12','13-16','17-20','21-24'],
    'AE_HRG':['Nothing','Low','Medium','High']
    }
# Create a function that uses a dictionary to order the ordinal columns
def order_categorical(df, column_dict):
    for column in column_dict:
        df[column] = df[column].cat.set_categories(column_dict[column], ordered=True)
# Order the ordinal categorical data
order_categorical(aedata_df,ordinal_columns_dict)

In [96]:
# Check the ordinal columns
for column in ordinal_columns:
    print(aedata_df[column])

0         2
1         2
2         3
3         1
4         2
         ..
326439    1
326440    3
326441    3
326442    2
326443    2
Name: IMD_Decile_From_LSOA, Length: 326444, dtype: category
Categories (10, int64): [1 < 2 < 3 < 4 ... 7 < 8 < 9 < 10]
0          1-17
1         65-84
2          1-17
3           85+
4          1-17
          ...  
326439     1-17
326440    65-84
326441    25-44
326442    45-64
326443    65-84
Name: Age_Band, Length: 326444, dtype: category
Categories (6, object): ['1-17' < '18-24' < '25-44' < '45-64' < '65-84' < '85+']
0         17-20
1         13-16
2         17-20
3         05-08
4         13-16
          ...  
326439    21-24
326440    21-24
326441    01-04
326442    17-20
326443    13-16
Name: AE_Arrive_HourOfDay, Length: 326444, dtype: category
Categories (6, object): ['01-04' < '05-08' < '09-12' < '13-16' < '17-20' < '21-24']
0         Nothing
1             Low
2             Low
3             Low
4             Low
           ...   
326439     Medium

In [97]:
# 4. Check the rows that have null values
print(aedata_df[aedata_df.isna().any(axis=1)])

       IMD_Decile_From_LSOA Age_Band Sex AE_Arrive_Date AE_Arrive_HourOfDay  \
42                        5    65-84   1     2017-01-05               05-08   
90                        3    65-84   2     2018-01-09               13-16   
190                       1     1-17   1     2017-03-28               05-08   
231                       2     1-17   2     2016-07-05               09-12   
289                       1     1-17   1     2015-08-26               09-12   
...                     ...      ...  ..            ...                 ...   
326188                    2    25-44   2     2016-01-30               13-16   
326324                    6    65-84   2     2016-09-03               09-12   
326343                    5    45-64   2     2018-03-18               09-12   
326358                    4    25-44   1     2015-04-12               13-16   
326364                    2    65-84   2     2016-03-24               21-24   

        AE_Time_Mins   AE_HRG  AE_Num_Diagnoses  AE

What percentage of the rows in the dataset have missing values?

In [98]:
# Compute the percentage of rows that have missing values
print(f"{round(len(aedata_df[aedata_df.isna().any(axis=1)])/len(aedata_df)*100,2)} % of rows have a null value.")

1.44 % of rows have a null value.


Theoretically, these null values would need to be farther assessed. Weather the null values are missing in a systematic way or purely random, weather the missing values could be imputed etc. But in the interest of time, I decided to remove the rows that have null values.

In [99]:
# Remove rows that have null values
aedata_df.dropna(axis=0,inplace=True)
# Check again the info of the working dataframe
aedata_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 321748 entries, 0 to 326443
Data columns (total 17 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   IMD_Decile_From_LSOA             321748 non-null  category      
 1   Age_Band                         321748 non-null  category      
 2   Sex                              321748 non-null  category      
 3   AE_Arrive_Date                   321748 non-null  datetime64[ns]
 4   AE_Arrive_HourOfDay              321748 non-null  category      
 5   AE_Time_Mins                     321748 non-null  float64       
 6   AE_HRG                           321748 non-null  category      
 7   AE_Num_Diagnoses                 321748 non-null  int64         
 8   AE_Num_Investigations            321748 non-null  int64         
 9   AE_Num_Treatments                321748 non-null  int64         
 10  AE_Arrival_Mode                  321748 non-

### What is the structure of your dataset?

> There are 321,749 A&E patient consult in the working dataset with 17 features. Two are date or period data, 6 are numeric variables, 

### What is/are the main feature(s) of interest in your dataset?

> Your answer here!

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

> Your answer here!

## Univariate Exploration

> In this section, investigate distributions of individual variables. If
you see unusual points or outliers, take a deeper look to clean things up
and prepare yourself to look at relationships between variables.


> **Rubric Tip**: The project (Parts I alone) should have at least 15 visualizations distributed over univariate, bivariate, and multivariate plots to explore many relationships in the data set.  Use reasoning to justify the flow of the exploration.



>**Rubric Tip**: Use the "Question-Visualization-Observations" framework  throughout the exploration. This framework involves **asking a question from the data, creating a visualization to find answers, and then recording observations after each visualisation.** 




>**Rubric Tip**: Visualizations should depict the data appropriately so that the plots are easily interpretable. You should choose an appropriate plot type, data encodings, and formatting as needed. The formatting may include setting/adding the title, labels, legend, and comments. Also, do not overplot or incorrectly plot ordinal data.

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Your answer here!

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> Your answer here!

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

## Conclusions
>You can write a summary of the main findings and reflect on the steps taken during the data exploration.



> Remove all Tips mentioned above, before you convert this notebook to PDF/HTML


> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML or PDF` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!

